In [54]:
import numpy as np
import pandas as pd
from sklearn.manifold import MDS, TSNE
import plotly.express as px

# Charger la matrice de distances depuis un fichier CSV
file_path = 'jaccardIndex/jaccard_clean_COMP.csv'  
distance_matrix = pd.read_csv(file_path)

# Nettoyage de la matrice de distances
distance_matrix.set_index('Unnamed: 0', inplace=True)
distance_matrix = distance_matrix.astype(float)

# Vérification si la matrice est carrée et symétrique
assert distance_matrix.shape[0] == distance_matrix.shape[1], "La matrice doit être carrée."
assert np.allclose(distance_matrix, distance_matrix.T), "La matrice doit être symétrique."

# Fonction pour effectuer l'embedding et afficher les résultats
def visualize_embedding(distance_matrix, n_dimensions=2, method="MDS"):
    mds = MDS(n_components=n_dimensions, dissimilarity='precomputed', metric=False, normalized_stress=True, random_state=42)
    embedding = mds.fit_transform(distance_matrix.values)
    stress = mds.stress_


    # Si 2D, générer une visualisation
    if n_dimensions == 2:
        title = f"Projection en 2D avec {method}"
        if stress is not None:
            title += f" (Stress: {stress:.2f})"

        fig = px.scatter(
            x=embedding[:, 0],
            y=embedding[:, 1],
            text=distance_matrix.index,
            labels={'x': 'Dimension 1', 'y': 'Dimension 2'},
            title=title
        )
        fig.update_traces(textposition='top center')
        fig.show()

    return embedding, stress

# Appel de la fonction pour une projection en 2D avec MDS
embedding_mds, stress_mds_2d = visualize_embedding(distance_matrix, n_dimensions=250)
print("Stress MDS:", stress_mds_2d)


Stress MDS: 0.0240561684614573


In [50]:

# Utiliser la sortie de MDS comme entrée pour t-SNE
def tsne_from_mds(embedding_mds, n_dimensions=2):
    tsne = TSNE(n_components=n_dimensions, random_state=42, init="random")
    embedding_tsne = tsne.fit_transform(embedding_mds)
    
    # Générer une visualisation
    fig = px.scatter(
        x=embedding_tsne[:, 0],
        y=embedding_tsne[:, 1],
        title="Projection en 2D avec t-SNE (à partir de MDS)",
        labels={"x": "Dimension 1", "y": "Dimension 2"}
    )
    fig.show()
    
    return embedding_tsne

# Appliquer t-SNE sur les données MDS
embedding_tsne_2d = tsne_from_mds(embedding_mds, n_dimensions=2)

In [17]:
import pandas as pd
import json

# Charger les données
data_file = 'data/cleanScopus/clean_VETE.csv'  # Remplace par le chemin de ton fichier
data = pd.read_csv(data_file)

def get_common_authors(issn1, issn2):
    # Filtrer les articles des deux ISSN
    articles_issn1 = data[data['prism:issn'] == issn1]
    articles_issn2 = data[data['prism:issn'] == issn2]

    # Extraire les auteurs (sans décoder le JSON)
    authors_issn1 = set(articles_issn1['author_info'])
    authors_issn2 = set(articles_issn2['author_info'])

    # Trouver les auteurs communs
    common_authors = authors_issn1 & authors_issn2

    # Calculer la proportion d'auteurs communs
    total_authors = authors_issn1 | authors_issn2
    proportion_common = len(common_authors) / len(total_authors) if total_authors else 0

    return list(common_authors), proportion_common

# Exemple d'utilisation
issn1 = '17922720'  
issn2 = '01613499'  
common_authors, proportion = get_common_authors(issn1, issn2)

print(f"Auteurs en commun : {common_authors}")
print(f"Proportion d'auteurs en commun : {proportion:.2%}")




Auteurs en commun : []
Proportion d'auteurs en commun : 0.00%
